In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import librosa
import librosa.display

from keras.models import Sequential
from keras.layers import Flatten, Dense

In [ ]:
data_dir = './tensorflow-speech-recognition-challenge/'

train_dir = os.path.join(data_dir, 'train/audio')
test_dir = os.path.join(data_dir, 'test/audio')

# Visualiseer de data

Laten we eerst eens naar een enkel bestand kijken waarin het woord 'cat' wordt uitgesproken.

In [ ]:
example_filename = os.path.join(train_dir, 'cat/00b01445_nohash_0.wav')

In [ ]:
samples_cat, sample_rate = librosa.load(example_filename)

librosa.display.waveshow(samples_cat, sr = sample_rate)

In [ ]:
len(samples_cat)

Ter vergelijking bekijken we een opname van het woord 'dog'.

In [ ]:
example_filename = os.path.join(train_dir, 'dog/0a7c2a8d_nohash_0.wav')

samples_dog, sample_rate = librosa.load(example_filename)

librosa.display.waveshow(samples_dog, sr = sample_rate)

## Spectrum

We plotten de frequenties in een klein stuk van de opname van 'cat'.

In [ ]:
n_fft = 2048
plt.plot(np.abs(librosa.stft(samples_cat[:n_fft], n_fft=n_fft, hop_length = n_fft + 1)))

## Spectogram

Een spectogram laat de geluidsintensiteit op verschillende frequenties uitgezet in de tijd zien.

In [ ]:
#cat

hop_length = 512

stft = np.abs(librosa.stft(samples_cat, n_fft = n_fft, hop_length = hop_length))
DB = librosa.amplitude_to_db(stft, ref=np.max)

librosa.display.specshow(DB, sr = sample_rate, x_axis = 'time', y_axis = 'log');
plt.colorbar(format='%+2.0f dB')

In [ ]:
#dog

stft = np.abs(librosa.stft(samples_dog, n_fft = n_fft, hop_length = hop_length))
DB = librosa.amplitude_to_db(stft, ref=np.max)

librosa.display.specshow(DB, sr = sample_rate, x_axis = 'time', y_axis = 'log');
plt.colorbar(format='%+2.0f dB')

## MFCC

MFCC features proberen de belangrijkste informatie in een spraaksignaal te representeren.

In [ ]:
mfccs = librosa.feature.mfcc(samples_cat, sr = sample_rate, n_mfcc = 13)
librosa.display.specshow(mfccs, x_axis='time')

# Herkennen van woorden

We kunnen de MFCC representatie gebruiken als input voor een deep learning model. We gaan proberen om een aantal verschillend woorden te herkennen met behulp van een LSTM. 

Je kunt eventueel meer labels toevoegen (namen van de mappen in de dataset).

In [ ]:
labels = ['cat', 'dog', 'yes', 'no']

## Prepareer de data

We zetten alle opnamen van de betreffende woorden om in MFCC features. We kiezen hier voor 13 MFCC features die we elke 10 ms berekenen voor een stukje van 25 ms van het spraaksignaal.

In [ ]:
X_train = []
y_train = []

max_len = 80 # om ons model iets te vereenvoudingen gebruiken we een vaste lengte voor de inputs


for label in labels:
    wav_dir = os.path.join(train_dir, label)
    waves = [f for f in os.listdir(wav_dir) if f.endswith('.wav')]
    
    for wav in waves:
        samples, sample_rate = librosa.load(os.path.join(wav_dir, wav), sr = 16000)
        mfccs = librosa.feature.mfcc(samples, sr = sample_rate, n_mfcc = 13, hop_length=int(sample_rate/100), n_fft=int(sample_rate/40))
        
        if (mfccs.shape[1] > max_len): # sla opnamen die te kort zijn over
            mfccs = mfccs[:, :max_len]
        
            X_train.append(mfccs)
            y_train.append(label)

We brengen X_train en y_train in de juiste vorm om gebruikt te worden door `keras` modellen.

In [ ]:
X_train = np.array(X_train)
X_train = np.swapaxes(X_train, 1, 2)
X_train.shape

In [ ]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(np.array(y_train))

y_train = to_categorical(y_train)

y_train[0:3]

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [ ]:
X_train.shape

In [ ]:
y_train.shape

## Selecteer een model, train, verbeter, evalueer

Experimenteer met geschikte Deep Learning modellen. Welk model werkt het beste?

Tip 1: de input heeft de vorm: `input_shape = (80, 13)`.

Tip 2: Dropout kan helpen.

Zijn er woorden die de modellen vaak door elkaar halen?

Bonusvraag: Hierboven hebben we plaatjes gemaakt van de data. We zouden deze afbeeldingen als input kunnen beschouwen. We zien iedere input dan dus als een twee-dimensionale afbeelding en kunnen deze classificeren met modellen voor beeldherkenning. Maak zo'n model.

Het is dan wel nodig om de vorm van `X_train` en `X_val` zo aan te passen dat deze een 'kleuren kanaal' hebben. Je kunt dit op de MFCC features doen, maar ook op een spectogram. gebruik hiervoor `librosa.melspectogram`.